In [ ]:
import plotly.express as px

In [ ]:
import json
with open('./geojson/서울_행정동_경계_2017.geojson') as j:
    seoul_geo = json.load(j)

In [ ]:
import pandas as pd
from os import listdir

bigdf = pd.DataFrame()

filenames = [
    "../참고 데이터/csv/large-datasets/2022/생활이동_행정동_202201/" + f
    for f in listdir("../참고 데이터/csv/large-datasets/2022/생활이동_행정동_202201/")
]

for file in filenames[:]:
    with open(file, encoding='cp949') as f:
        data = pd.read_csv(f)

    data = data.drop(data[data['출발 행정동 코드'] == data['도착 행정동 코드']].index)
    # data.loc[data['이동인구(합)'] == '*', '이동인구(합)'] = 0/
    data['이동인구(합)'] = data['이동인구(합)'].replace(to_replace="*", value=0)
    data['이동인구(합)'] = data['이동인구(합)'].astype(float)

    data = data.drop(data[(data['출발 행정동 코드'] < 1101053)].index)
    data = data.drop(data[(data['출발 행정동 코드'] > 1125074)].index)
    data = data.drop(data[(data['도착 행정동 코드'] < 1101053)].index)
    data = data.drop(data[(data['도착 행정동 코드'] > 1125074)].index)

    data2point5 = pd.DataFrame(data.groupby(['도착 행정동 코드'])['이동인구(합)'].sum())
    data2point5 = data2point5.reset_index()
    # data2point5['도착 행정동 코드'] = str(data2point5['도착 행정동 코드'])
    # data2point5.sort_values('도착 행정동 코드')
    data2point5['도착 행정동 코드'] = data2point5['도착 행정동 코드'].astype(str)

    new_row = data2point5
    new_row['time'] = int(file.split("/")[6].split("_")[3][0:2])
    bigdf = pd.concat([bigdf, new_row])

bigdf

In [ ]:
fig = px.choropleth_mapbox(bigdf,
                           geojson=seoul_geo,
                           locations='도착 행정동 코드',
                           color='이동인구(합)',
                           color_continuous_scale="Viridis",
                           featureidkey="properties.adm_cd",
                           mapbox_style="carto-positron",
                           opacity=0.5,
                           zoom=10,
                           center={
                               "lat": 37.559,
                               "lon": 126.963
                           },
                           labels={'이동인구(합)':'label'},
                           animation_frame='time'
                           )

fig.write_html('../작업 결과물/행정동_시간별지도(2022년1월).html')
# fig.show() # 파일이 너무 커집니다!!